# Training Time

In [17]:
import glob
from pprint import pprint
import pickle
import re
import sys

import numpy as np
import tensorflow as tf
from tensorflow.keras import layers
from tensorflow.keras import backend as K

sys.path.insert(0, "/home/jupyter/metisproject5/notebooks/")
from slice_generator import SliceGenerator

In [ ]:
print("tensorflow version:",tf.__version__)

## Model variables

In [2]:
frames = 1
channels = 1
pixels_x = 21
pixels_y = 21

## get and save the pwd

In [3]:
model_dir = "."
models_avail = sorted(glob.glob(model_dir+"*"))
print("available models:")
pprint(models_avail)

available models:
['.ipynb_checkpoints', '.less_deconv_24F.h5']


In [26]:
model_folder = "/"
pwd = model_dir+model_folder
file_list = sorted(glob.glob(pwd+"*"))
print("Files in folder:")
pprint(file_list)

Files in folder:
['./02_lessdeconv.ipynb', './03_train.ipynb', './04_extract_prediction.ipynb']


## Find and load model

In [6]:
# choose a model
models=[]
for file in file_list:
    match = re.search("\.h5$", file)
    if match:
        models.append(file)

assert len(models) > 0, "Hmm, can't seem to find any model files ending in .h5 in "+pwd
if len(models) > 1:
    print("There are more than one models in the directory.\nLoading the first.\n", models)
print("Loading... ")
model = tf.keras.models.load_model(models[0])

model_name = re.search("[^/]*(?=\.[^.]+($|\?))", models[0]).__getitem__(0)
print("Loading done for",model_name)

Loading... 
Loading done for .less_deconv_24F


In [7]:
# model_name += "_"+loss_func_name
# model_name

# Loss, Learning Rate, Optimizer

## Loss
Define a custom loss function. Here it's SSE

In [7]:
loss_func_name = "SSE"
def sum_sq_err(y_true, y_pred):
    return K.sum( K.square(y_true - y_pred), axis=1)
class SumSquaredError(tf.keras.losses.Loss):
    def __init__():
        self.name = "SSE"
    def call(self, y_true, y_pred):
#         y_pred = ops.convert_to_tensor_v2(y_pred)
#         y_true = math_ops.cast(y_true, y_pred.dtype)
        return K.sum( K.square(y_true - y_pred), axis=1)

## lr
Define a Learning rate schedule.

In [8]:
lr = 0.01
# lr = tf.keras.optimizers.schedules.ExponentialDecay(
#     initial_learning_rate = 1./3.1415,
#     decay_steps = 1,
#     decay_rate = 0.367,
#     staircase=False,
#     name="inversePi_e_inverse7"
# )

## optimizer
Define an optimizer. Must set ```clipnorm``` to prevent wild things from happening

In [9]:
opt = tf.keras.optimizers.SGD(
    learning_rate=0.01,
    momentum=0.9,
    nesterov=False,
    name='SGD',
    clipnorm=1.0,
)
# tf.keras.optimizers.Adam(learning_rate=lr,
#                                beta_1=0.9,
#                                beta_2=0.999,
#                                epsilon=1e-07,
#                                amsgrad=False,
#                                name='Adam',
#                                clipnorm=1.0)

# Compile the Model

In [10]:
print("Compiling", model_name,"...")
model.compile(
    loss=sum_sq_err,
    optimizer=opt,
    metrics=['mean_absolute_error'])
print("Compiled.")

Compiling .less_deconv_24F ...
Compiled.


# Prepare for training, callbacks
Get paths to training and validation sets, and number of steps

In [12]:
train_file_path = "../../data/train"
valid_file_path = "../../data/validate"
vars_           = ['t2m']
proc_type       = "convlstm"
# 3 years of training data = 
train_steps = 3 * 365 * 24 / frames
# 1 year of validation data = 
valid_steps = 1 * 365 * 24 / frames

## callbacks
Define callbacks to periodically save the model, stop early, write logs to tensorboard, and stream loss metrics to a csv file

In [13]:
callbacks = [
    tf.keras.callbacks.ModelCheckpoint(
        filepath=pwd+model_name+".h5", 
        monitor='val_loss',
        verbose=1,
        save_best_only=True,
        save_weights_only=False,
        mode='auto',
        save_freq='epoch',
    ),
    tf.keras.callbacks.EarlyStopping(
        # Stop training when `val_loss` is no longer improving
        monitor="val_loss",
        # "no longer improving" being defined as "no better than 1e-2 less"
        min_delta=1e-2,
        # "no longer improving" being further defined as "for at least 1 epochs"
        patience=1,
        verbose=1,
    ),
    tf.keras.callbacks.CSVLogger(
        filename=pwd+"log.log",
        separator=',',
        append=True
    ),
]

# Train and save

In [14]:
epochs = 1

In [ ]:
model.fit(
    SliceGenerator(img_dir=train_file_path, slice_size=frames, 
                    vars_=vars_, proc_type=proc_type,
                    pixels_x=pixels_x, pixels_y=pixels_y, debug=False
                    ),
    steps_per_epoch = train_steps,
    epochs = epochs,
    verbose = 1,
    shuffle = False,
    callbacks=callbacks,
    validation_steps = valid_steps,
    validation_data = SliceGenerator(img_dir=valid_file_path, slice_size=frames,
                                      vars_=vars_, proc_type=proc_type,
                                      pixels_x=pixels_x, pixels_y=pixels_y, debug=False
                                      ),
)

tf.keras.models.save_model(
        model = model,
        filepath = pwd+model_name+'.h5',
        overwrite=True,
        include_optimizer=True,
        save_format='tf',
        signatures=None
)


13028/26280 [=============>................] - ETA: 11:14 - loss: 24.7694 - mean_absolute_error: 3.5836

## Save the model
Store history, save plot of loss

In [31]:
print(pwd+model_name+'.h5'
      
)
print(pwd)

./.less_deconv_24F.h5
./


In [28]:
tf.keras.models.save_model(
        model = model,
        filepath = pwd+model_name+'.h5',
        overwrite=True,
        include_optimizer=True,
        save_format='tf',
        signatures=None
)

## Store the history!

In [17]:
model_history = model.history.history

In [27]:
print("model history:")
pprint(model_history)
pickle.dump(model_history, open(pwd+model_name+"_hist.pkl", "wb"))
print("Model history saved to:",pwd+model_name+"_hist.pkl")

model history:


NameError: name 'model_history' is not defined

## Plot loss over epochs

In [21]:
import matplotlib.pyplot as plt

train_loss = model_history["loss"]
val_loss = model_history["val_loss"]
plt.figure(figsize=(8, 5))
plt.title("Training vs. Validation Loss")
plt.plot(train_loss, label="Training Loss")
plt.plot(val_loss, label="Validation Loss")
plt.xlabel("Number of Epochs", size=14)
plt.legend()
plt.show()
plt.savefig(pwd+model_name+"_loss.png",transparent=True, format="png")

NameError: name 'model_history' is not defined

## Visually Inspect Prediction

In [22]:
slice_val = SliceGenerator(img_dir=train_file_path, slice_size=frames, vars_=vars_,
                     proc_type=proc_type, pixels_x=pixels_x, pixels_y=pixels_y, debug=False
                     )

In [23]:
in_, out_ = next(slice_val)
in_

array([[[[283.6044 , 283.62558, 283.63556, 283.03384, 283.1821 ,
          282.90118, 281.11227, 277.90506, 276.03955, 277.31894,
          281.29352, 282.79776, 282.401  , 278.17853, 273.87567,
          270.05182, 269.29565, 268.20187, 267.5653 , 267.3093 ,
          267.73535],
         [283.5826 , 283.4792 , 283.71777, 283.3291 , 283.2232 ,
          283.61996, 282.5181 , 279.49902, 276.1143 , 278.43637,
          281.12784, 283.40695, 284.26028, 282.29385, 278.58276,
          274.04758, 270.36017, 268.43796, 267.14923, 267.1274 ,
          268.24298],
         [283.6704 , 283.57324, 283.48917, 283.53027, 283.7408 ,
          284.11392, 282.97342, 281.36765, 279.68213, 278.86618,
          280.8407 , 283.14532, 284.7779 , 284.3855 , 282.657  ,
          280.15305, 275.53378, 271.1101 , 267.38715, 265.74463,
          266.08658],
         [283.6804 , 283.6804 , 283.40134, 283.5533 , 283.8679 ,
          284.51816, 283.53775, 282.3599 , 281.37762, 279.6728 ,
          281.09045, 282

In [24]:
test_pred= model.predict(in_, verbose=1)
test_pred

1/1 [==============================] - 0s 3ms/step


array([[[[[  281.60672,   281.60672,   281.60672,   281.60672,
             281.60672,   281.60672,   281.60672,   281.60672,
             281.60672,   281.60672,   281.60672,   281.60672,
             281.60672,   281.60672,   281.60672,   281.60672,
             281.60672,   281.60672,   281.60672,   281.60672,
             281.60672],
          [  281.60672,   281.60672,   281.60672,   281.60672,
             281.60672,   281.60672,   281.60672,   281.60672,
             281.60672,   281.60672,   281.60672,   281.60672,
             281.60672,   281.60672,   281.60672,   281.60672,
             281.60672,   281.60672,   281.60672,   281.60672,
             281.60672],
          [  281.60672,   281.60672,   281.60672,   281.60672,
             281.60672,   281.60672,   281.60672,   281.60672,
             281.60672,   281.60672,   281.60672,   281.60672,
             281.60672,   281.60672,   281.60672,   281.60672,
             281.60672,   281.60672,   281.60672,   281.60672,
     